# Singapore Public Housing (HDB) Resale Price Prediction Model (Part 6)
### Feature Engineering - Hexagon Map

## 1. Initialization

In [1]:
# # Dependencies for Hexagon-Map Creation
# # No need to import
# !apt install libspatialindex-dev
# !pip install rtree, geopandas

In [2]:
# Vanilla packages
import numpy as np
import pandas as pd
import math

import geopandas as gpd
from shapely.geometry import Point
from shapely import wkt
from matplotlib.patches import RegularPolygon

pd.set_option('max_columns', 99)

In [3]:
hdb = pd.read_csv('./Dataset/Transitional/final_data.csv')

## 2. Hexagon Map Data Preparation

In [4]:
# Function to calculate haversine distance between two coordinates
def haversine(coord1, coord2):
    lon1, lat1 = coord1
    lon2, lat2 = coord2
    
    # Earth radius
    R = 6_371_000
    phi_1 = np.radians(lat1)
    phi_2 = np.radians(lat2)
    delta_phi = np.radians(lat2 - lat1)
    delta_lambda = np.radians(lon2 - lon1)
    a = np.sin(delta_phi / 2.0) ** 2 + np.cos(phi_1) * np.cos(phi_2) * np.sin(delta_lambda / 2.0) ** 2
    c = 2 * np.arctan2(np.sqrt(a),np.sqrt(1 - a))
    meters = R * c
    km = meters / 1000.0
    meters = round(meters)
    km = round(km, 3)
    return meters

In [5]:
# ### ---      KNOWN DEPENDENCIES ISSUE      --- ###
# ### ---  CODES TO EXPORT FOR GOOGLE COLAB  --- ###
# ### ---     UNCOMMENT TO TEST THE CODE     --- ###

# # Loading data using Geopandas
# df = gpd.read_file('./Raw/planning_area_with_whampoa.json')

# # Drop off planning area that are not HDB estate to save up processing power and memory
# df = df.drop([0, 1, 2, 3, 4, 5, 7, 8, 10, 11, 13, 14, 15, 16, 18, 19, 25, 26, 29, 30, 34, 39, 44, 51, 53]).reset_index(drop=True)

# # Combining smaller planning areas in accordance of HDB estate neighbourhood in raw DataFrame
# df.loc[[0, 25, 26, 29, 30], 'name'] = 'CENTRAL AREA'
# df.loc[[1, 31], 'name'] = 'KALLANG/WHAMPOA'

# # Dissolve area with the same name
# TO_boundary = df.dissolve(by='name')

# # Export to geojson file for further processing
# TO_boundary.to_file('./Dataset/Spatial/planning_area_optimized.geojson', driver='GeoJSON')

In [6]:
def create_hex_map(hex_diam=250):
    # Import the optimized planning area polygons
    df = gpd.read_file('./Dataset/Spatial/planning_area_optimized.geojson')

    # Furthest coordinates on Singapore
    xmin, ymin, xmax ,ymax = df.total_bounds

    # East-West & North-South Length of Singapore
    EW = haversine((xmin, ymin), (xmax, ymin))
    NS = haversine((xmin, ymin), (xmin, ymax)) + 10000

    # Diameter of hexagon in meters
    d = hex_diam

    # Calculate width of hexagon in meters and grid structure of hexagon
    w = d * np.sin(np.pi / 3)
    n_cols = int(EW / w) + 1
    n_rows = int(NS / d) + 1

    # Calculate width of hexagon in coordinates
    w = (xmax - xmin) / n_cols

    # Calculate diameter of hexagon in coordinates
    d = w / np.sin(np.pi / 3)

    array_of_hexes = []

    for rows in range(0, n_rows):
        hcoord = np.arange(xmin, xmax, w) + (rows % 2) * w / 2
        vcoord = [ymax - rows * d * 0.75] * n_cols
        for x, y in zip(hcoord, vcoord):
            hexes = RegularPolygon((x, y), numVertices=6, radius=d/2, alpha=0.2, edgecolor='k')
            verts = hexes.get_path().vertices
            trans = hexes.get_patch_transform()
            points = trans.transform(verts)
            array_of_hexes.append(Polygon(points))

    hex_grid = gpd.GeoDataFrame({'geometry': array_of_hexes}, crs={'init': 'epsg:4326'})
    TO_hex = gpd.overlay(hex_grid, df)
    TO_hex = gpd.GeoDataFrame(TO_hex,geometry='geometry')
    TO_hex.to_file('./Dataset/Spatial/planning_area_hexed.geojson', driver='GeoJSON')
    TO_hex[['geometry']].to_csv('./Dataset/Spatial/HexMap.csv')

In [7]:
# ### ---      KNOWN DEPENDENCIES ISSUE      --- ###
# ### ---  CODES TO EXPORT FOR GOOGLE COLAB  --- ###
# ### ---     UNCOMMENT TO TEST THE CODE     --- ###

# create_hex_map()

In [8]:
# Load in hex-ed singapore map into Geopandas DataFrame
hex_map = pd.read_csv('./Dataset/Spatial/HexMap.csv')
hex_map['geometry'] = hex_map['geometry'].apply(wkt.loads)
hex_map = gpd.GeoDataFrame(hex_map, geometry='geometry')

## 3. Feature Engineering

In [9]:
# Remove terrace-house (outliers) from dataset for better mapping visualization
hdb = hdb[hdb['flat_model']!='Terrace'].reset_index(drop=True)

# Price per square feet
hdb['price_per_sqft'] = hdb['resale_price'] / hdb['floor_area_sqm'] * 10.7639

# Unit Price --- Price per area per year lease
hdb['unit_price'] = hdb['price_per_sqft'] / hdb['remaining_lease']

# Elementary Price --- Price per area per year lease per storey level
hdb['elem_price'] = hdb['unit_price'] / hdb['storey_range']

In [10]:
# # Original method of grouping property entries by address
# hex_df = hdb.groupby('address').mean().reset_index()

# Renewed method to bypass calculation running
hex_df = pd.read_csv('./Dataset/final_data_with_hex.csv')

# Label each address to their respective hexagon
if hex_df['hex_id'].isnull().sum() > 0:
    for idx, (latitude, longitude) in enumerate(zip(hex_df['latitude'], hex_df['longitude'])):
        print('\rWaiting... {} addresses remaining... '.format(len(hex_df)-idx-1), end='.')
        if math.isnan(hex_df.loc[idx, 'hex_id']):
            for boundary, hex_id in zip(hex_map.geometry, hex_map.hex_id):
                coor = Point(longitude, latitude)
                if coor.within(boundary):
                    hex_df.loc[idx, 'hex_id'] = hex_id
                    break
            hex_df.to_csv('./Dataset/final_data_with_hex.csv', index=False)

else:
    print('--- DATA IS COMPLETE ---')

--- DATA IS COMPLETE ---


In [13]:
# Further Optimize by taking only hexagon with address/data
hex_map.columns = ['hex_id', 'geometry']
hex_map = hex_map[hex_map['hex_id'].isin(hex_df['hex_id'].unique())]

# ### ---      KNOWN DEPENDENCIES ISSUE      --- ###
# ### ---  CODES TO EXPORT FOR GOOGLE COLAB  --- ###
# ### ---     UNCOMMENT TO TEST THE CODE     --- ###

# # Export for Colab
# hex_map.to_csv('./Dataset/Spatial/HexMap_Optimized.csv', index=False)

# # Read from Colab
# hex_map = pd.read_csv('./Dataset/Spatial/HexMap_Optimized.csv')
# hex_map['geometry'] = df['geometry'].apply(wkt.loads)
# hex_map = gpd.GeoDataFrame(df, geometry='geometry')

# # Export the optimized hexmap to geojson format for mapping purpose
# hex_map.columns = ['hex_id', 'geometry']
# hex_map.to_file('./Dataset/Spatial/hexmap.geojson', driver='GeoJSON')

In [14]:
# Remove any missing points not hex-ed (outliers far from actual HDB estate)
# Only 3 addresses affect - 1 x Changi Village + 2 x Farrer Road
hex_df = hex_df.dropna().reset_index(drop=True)

# Only merged if column not exist - to ease re-running of code
if 'town' not in hex_df.columns:
    hex_df = pd.merge(hex_df, hdb[['address', 'town']], how='left', on='address')
    
hex_df = hex_df.drop_duplicates().reset_index(drop=True)

In [15]:
# Aggregate HDB-town-wide local Minimum and Maximum
town_ave = hex_df.groupby('town').agg(['min', 'max'])['unit_price'].reset_index()
town_ave.columns = ['town', 'town_min_unit_price', 'town_max_unit_price']

# Only merged if column not exist - to ease re-running of code
if 'town_min_unit_price' not in hex_df.columns:
    hex_df = pd.merge(hex_df, town_ave, how='left', on='town')

In [16]:
# Normalize price of property with respect to its HDB Township
max_ = hex_df['town_max_unit_price']
min_ = hex_df['town_min_unit_price']
hex_df['appeal_score'] = (hex_df['unit_price'] - min_) / ( max_ - min_)

## 4. Final Data Export

In [18]:
hex_df.to_csv('./Dataset/final_data_with_hex.csv', index=False)